# 데이터 전처리

In [22]:
import glob
import tensorflow as tf
import os, re
import numpy as np

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'
txt_list = glob.glob(txt_file_path)

raw_corpus=[]

for txt_file in txt_list:
    with open(txt_file,'r') as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)
        
print("데이터 크기:",len(raw_corpus))
print("Examples:\n",raw_corpus[:3])

데이터 크기: 187088
Examples:
 ["Now I've heard there was a secret chord", 'That David played, and it pleased the Lord', "But you don't really care for music, do you?"]


파이썬에서 필요한 라이브러리를 불러들리고 분석할 데이터를 한 줄씩 읽어 raw_corpus에 저장 

In [23]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)
    sentence = sentence.strip()
    sentence = '<start> '+ sentence + ' <end>'
    return sentence

raw_corpus의 문장에서 데이터를 정리하기 위해 소문자로 바꾸고, 양쪽 공백을 지움. 특수문자 양쪽에 공백을 넣고 여러개의 공백은 하나의 공백으로 바꿈. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꾸고 다시 양쪽 공백을 지움. 그리고 마지막으로 문장 시작에는 '<start>' 끝에는 '<end>'를 추가함

In [24]:
corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0: continue
    if sentence[-1] == ':': continue
    
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
    
corpus[:10]

['<start> now i ve heard there was a secret chord <end>',
 '<start> that david played , and it pleased the lord <end>',
 '<start> but you don t really care for music , do you ? <end>',
 '<start> it goes like this <end>',
 '<start> the fourth , the fifth <end>',
 '<start> the minor fall , the major lift <end>',
 '<start> the baffled king composing hallelujah hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah your faith was strong but you needed proof <end>']

마지막으로 아무 것도 없는 문장이나 뒤에 ':'로 끝나는 문장들은 삭제

In [40]:
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words = 12000,
        filters = ' ',
        oov_token = "<unk>")
    
    tokenizer.fit_on_texts(corpus)
    tensor = tokenizer.texts_to_sequences(corpus)
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post',maxlen = 15)
    
    print(tensor, tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2   50    5 ...    0    0    0]
 [   2   17 2639 ...    0    0    0]
 [   2   36    7 ...   43    3    0]
 ...
 [   5   22    9 ...   10 1013    3]
 [  37   15 9049 ...  877  647    3]
 [   2    7   34 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f6b644bca00>


단어 사전을 만들기 위한 토큰화 함수로 12000단어를 저장

In [41]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])
    
    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to


In [42]:
src_input = tensor[:, :-1]
tgt_input = tensor[:, 1:]

print(src_input[0])
print(tgt_input[0])

[   2   50    5   91  297   65   57    9  969 6042    3    0    0    0]
[  50    5   91  297   65   57    9  969 6042    3    0    0    0    0]


# 데이터 학습

In [43]:
from sklearn.model_selection import train_test_split

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, 
                                                          tgt_input,
                                                          test_size=0.2, 
                                                          shuffle = True,
                                                          random_state = 2022)

print('train data: ', enc_train.shape)
print('target data: ', dec_train.shape)

train data:  (140599, 14)
target data:  (140599, 14)


train 데이터와 test 데이터를 생성

In [44]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__() 
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size) 
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)  
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
 
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size) 

TextGenerator라는 RNN 모델을 생성 총 1개의 Embedding Layer, 2개의 LSTM Layer, 그리고 1개의 Dense Layer로 구성됨

In [45]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True,
                                                    reduction = 'none'
                                                    )

In [46]:
model.compile(loss=loss, optimizer=optimizer)
model.fit(enc_train, dec_train, epochs=10)

Epoch 1/10
4394/4394 [==============================] - 160s 35ms/step - loss: 3.2862
Epoch 2/10
4394/4394 [==============================] - 156s 35ms/step - loss: 2.8366
Epoch 3/10
4394/4394 [==============================] - 155s 35ms/step - loss: 2.5624
Epoch 4/10
4394/4394 [==============================] - 156s 35ms/step - loss: 2.3185
Epoch 5/10
4394/4394 [==============================] - 156s 35ms/step - loss: 2.1021
Epoch 6/10
4394/4394 [==============================] - 156s 35ms/step - loss: 1.9103
Epoch 7/10
4394/4394 [==============================] - 156s 35ms/step - loss: 1.7408
Epoch 8/10
4394/4394 [==============================] - 155s 35ms/step - loss: 1.5911
Epoch 9/10
4394/4394 [==============================] - 155s 35ms/step - loss: 1.4626
Epoch 10/10
4394/4394 [==============================] - 155s 35ms/step - loss: 1.3537


# 데이터 평가

In [47]:
def generate_text(lyricist, tokenizer, init_sentence = "<start> i love", max_len = 20):
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype = tf.int64)
    end_token = tokenizer.word_index["<end>"]
    
    while True:
        predict = model(test_tensor)
        predict_word = tf.argmax(tf.nn.softmax(predict, axis = -1), axis = -1)[:, -1]
        
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis = 0)], axis = -1)
        
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break
            
    generated = ''
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "
        
    return generated

In [48]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len = 20)

'<start> i love you so , so , i do <end> '

1. padding = 'post', epoch = 30, embedding_size =  256, hidden_size = 1024로 실행하였을 때 loss는 epoch 10 이전에 2.2.로 내려갔지만 문장을 생성했을 때 '<start> I love it when you call me poppa <end>'라는 문장 생성

2. padding = 'pre', epoch = 10, embedding_size = 1024, hidden_size = 2048로 loss는 여전히 잘 내려가지만 문장이 '<start> I love you for in my life you are a friend of mine <end>'라는 문장을 생성하므로 문제가 있다는 것을 알 수 있음

3. padding = 'pre', epoch =10, embedding_size = 256, hidden_size = 1024로 다시 실행 loss는 여전히 잘 내려가지만 '<start> i love you slim , we coulda been together , we re jammin <end> '
이라는 알 수 없는 문장을 만듦

4. padding = 'post', epoch = 10, embedding_size = 256, hidden_size = 1024 로 실행 loss는 아무 문제없이 2.2이하를 잘 유지하며 문장은 '<start> i love you so , so , i do <end> '로 나름 감정이 많이 실린 문장을 생성함